In [1]:
import pandas as pd
from pathlib import Path

#### DEFINE CONTROL PARAMETERS 

In [2]:
# Select which features/columns to include in ML Model feature set
# Select which "derived" features to include in ML Model feature set

# Select crypto assets interested in evaluating
INCLUDE_BTEC = True
INCLUDE_ETH = True
INCLUDE_ADA = True

# Select the orderbook timeframe's interested in
INCLUDE_5MIN_DATA = True
INCLUDE_1MIN_DATA = False
INCLUDE_1SEC_DATA = False

# Optional removing of feature/columns from provided crypto order book
# Remove cancel features/columns
REMOVE_CANCEL_FEATURES = False
# Remove market features/columns
REMOVE_MARKET_FEATURES = False
# Remove price distance feature/column
REMOVE_PRICE_DISTANCES = False

# Optional NEW derived feature/columns that can be added to dataframe order book
# Option to add aboslute price levels to feature set (default dataframe shows delta price with mid price = distance)
ADD_ABSOLUTE_PRICE_LEVELS = True
# Option to add quantity at each level
ADD_QUANTITY_FEATURE = True
# Option to add CUMULATIVE quantity "across" levels
ADD_CUM_QUANTITY_FEATURE = True
# Option to add CUMULATIVE quantity "across" levels
ADD_CUM_NOTIONAL_FEATURE = True
# Option to add bid and ask IMBALANCE
ADD_BID_ASK_IMBALANCE = True
# ToDo: ADD MORE FEATURES (e.g. logarithmic returns)

# Level selection
# Select number of orderbook levels to include in feature set (max = 15)
MAX_ORDER_BOOK_LEVELS = 15   # Do not modify
NUM_ORDER_BOOK_LEVELS = 3

In [3]:
TURN_ON_PRINT_DEBUG = True

In [4]:
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_columns', 200)

#### Import the Crytpo Order Book Data

In [5]:
# dictionary to store crypto dataframes {BTEC, ETH, ADA}, {5min, 1min, 1sec}
crypto_ob_df_dict = {} 

# Based on the selection critero, read the csv crypto order book csv files into the dictrionary
if (INCLUDE_5MIN_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_5min"] = pd.read_csv(
            Path("./data/BTC_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_5min"] = pd.read_csv(
            Path("./data/ETH_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_5min"] = pd.read_csv(
            Path("./data/ADA_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    
if (INCLUDE_1MIN_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_1min"] = pd.read_csv(
            Path("./data/BTC_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_1min"] = pd.read_csv(
            Path("./data/ETH_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_1min"] = pd.read_csv(
            Path("./data/ADA_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    
if (INCLUDE_1SEC_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_1sec"] = pd.read_csv(
            Path("./data/BTC_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_1sec"] = pd.read_csv(
            Path("./data/ETH_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_1sec"] = pd.read_csv(
            Path("./data/ADA_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )    

In [6]:
# Debug data checkpoint

if (TURN_ON_PRINT_DEBUG):
    display(crypto_ob_df_dict.keys())
    
    for crypto_ob_key in crypto_ob_df_dict: 
        print(f"\nDebug {crypto_ob_key} dataframe after downloading/reading crypto provided csv files:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=5))

dict_keys(['BTC_5min', 'ETH_5min', 'ADA_5min'])


Debug BTC_5min dataframe after downloading/reading crypto provided csv files:


(3367, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,0,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,-1.294276e-04,-0.000130,-0.000143,-0.000201,-0.000201,-0.000240,-0.000271,-0.000387,-0.000399,-0.000424,-0.000588,-0.000588,-0.000629,-0.000727,18997.929688,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,13969.589844,3362.449951,27044.119141,2793.360107,25698.880859,2793.239990,500.000000,1.309910e+03,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,624.530029,0.00000,0.0,0.0,0.000000,0.000000,0.0,2.030784e+04,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,14594.120117,3362.449951,27044.119141,2793.360107,25698.880859,2793.239990,500.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,


Debug ETH_5min dataframe after downloading/reading crypto provided csv files:


(3365, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,0,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,-0.000129,-0.000200,-0.000246,-0.000261,-0.000297,-0.000317,-0.000454,-0.000505,-0.000510,-0.000515,-0.000530,-0.000535,-0.000550,-0.000566,1032.089966,4995.910156,29558.550781,197.050003,29556.750000,3241.280029,5911.020020,29298.880859,3901.620117,197.000000,984.979980,2669.790039,1969.910034,755.229980,3742.709961,0.000000,0.000000,0.000000,1673.790039,1673.760010,0.0,12750.349609,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,984.919983,1032.089966,4995.910156,29558.550781,1870.839966,31230.509766,3241.280029,18661.369141,29298.880859,3901.620117,197.000000,984.979980,2669.790039,1969.910034,755.229980,4727.640137,0.000000,0.000000,0.0,


Debug ADA_5min dataframe after downloading/reading crypto provided csv files:


(3367, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,0,1.17255,0.0009,330639.048591,500585.424671,-0.000384,-0.000469,-0.000810,-0.000981,-0.001066,-0.001663,-0.001748,-0.002089,-0.002175,-0.002260,-0.002431,-0.002601,-0.002772,-0.002857,-0.002942,2461.409912,2996.790039,1071.430054,20558.070312,1000.000000,1308.819946,10656.429688,3063.939941,29.250000,1997.170044,288.920013,804.619995,1389.130005,25513.240234,7818.939941,0.000000,0.00000,2996.979980,3474.47998,0.000000,0.000000,25248.980469,0.000000,0.00000,0.000000,1609.510010,0.0,2414.600098,13872.099609,1609.849976,2461.409912,2996.790039,4068.399902,24032.550781,1000.000000,1308.819946,35905.421875,3063.939941,29.250000,1997.170044,1898.420044,804.619995,3803.729980,39385.339844,9428.790039,0.0,0.0

#### PRE-PROCESS ORDER BOOK FEATURES/COLUMNS

In [7]:
# Drop the "Unnamed" column

for crypto_ob_key in crypto_ob_df_dict: 
    crypto_ob_df_dict[crypto_ob_key].drop("Unnamed: 0", axis=1, inplace=True)
    
    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after dropping Unnamed column:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))


Debug BTC_5min dataframe after dropping Unnamed column:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,-0.000129,-0.00013,-0.000143,-0.000201,-0.000201,-0.00024,-0.000271,-0.000387,-0.000399,-0.000424,-0.000588,-0.000588,-0.000629,-0.000727,18997.929688,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,13969.589844,3362.449951,27044.119141,2793.360107,25698.880859,2793.23999,500.0,1309.910034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,624.530029,0.0,0.0,0.0,0.0,0.0,0.0,20307.839844,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,14594.120117,3362.449951,27044.119141,2793.360107,25698.880859,2793.23999,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.944544e-08,0.000033,0.000278,0.000345,0


Debug ETH_5min dataframe after dropping Unnamed column:


(3365, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,-0.000129,-0.0002,-0.000246,-0.000261,-0.000297,-0.000317,-0.000454,-0.000505,-0.00051,-0.000515,-0.00053,-0.000535,-0.00055,-0.000566,1032.089966,4995.910156,29558.550781,197.050003,29556.75,3241.280029,5911.02002,29298.880859,3901.620117,197.0,984.97998,2669.790039,1969.910034,755.22998,3742.709961,0.0,0.0,0.0,1673.790039,1673.76001,0.0,12750.349609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,984.919983,1032.089966,4995.910156,29558.550781,1870.839966,31230.509766,3241.280029,18661.369141,29298.880859,3901.620117,197.0,984.97998,2669.790039,1969.910034,755.22998,4727.640137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000124,0.000129,0.000155,0.000332,0.000337,0


Debug ADA_5min dataframe after dropping Unnamed column:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,-0.000469,-0.00081,-0.000981,-0.001066,-0.001663,-0.001748,-0.002089,-0.002175,-0.00226,-0.002431,-0.002601,-0.002772,-0.002857,-0.002942,2461.409912,2996.790039,1071.430054,20558.070312,1000.0,1308.819946,10656.429688,3063.939941,29.25,1997.170044,288.920013,804.619995,1389.130005,25513.240234,7818.939941,0.0,0.0,2996.97998,3474.47998,0.0,0.0,25248.980469,0.0,0.0,0.0,1609.51001,0.0,2414.600098,13872.099609,1609.849976,2461.409912,2996.790039,4068.399902,24032.550781,1000.0,1308.819946,35905.421875,3063.939941,29.25,1997.170044,1898.420044,804.619995,3803.72998,39385.339844,9428.790039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000384,0.000725

In [8]:
# Rearrange columns in crypto dataframes
# The default arrangement of the order book data provided list all the levels for each feature in order of level
#     (e.g. bids_distance_0, bids_dstance_1, ..., bids_distance_14, bids_notional_0, bids_notional_1, ..., bids_notional_14, ... )
# Prefer to have all the same level features listed together in the columns (makes viewing of the dataframes easier)
#     (e.g. bids_distance_0, bids_notional_0, bids_price_0, asks_distance_0, ... bids_distance_1, ...)

# Keep the first 4 columns in the same position
new_coloumns_lst = ["midpoint", "spread", "buys", "sells"]

start_col_index = 4 # incrementing variable to index/location of where to move the columns to
bid_ask_str_lst = ["bids", "asks"] 
level_str_lst = ["distance", "limit_notional", "market_notional", "cancel_notional", "notional"]
# (Note: notional = limit_notional - market_notional - cancel_notional)

# Add the column strings to the list in the level order (e.g. bids_distance_0, bids_notional_0, bids_price_0, asks_distance_0, ... bids_distance_1, ...)
for level_num in range(MAX_ORDER_BOOK_LEVELS):
    for bid_ask_str in bid_ask_str_lst:
        for level_str in level_str_lst:
            column_name_str_to_move = bid_ask_str + "_" + level_str + "_" + str(level_num)
            new_coloumns_lst.append(column_name_str_to_move)

# Reorder the columns for each of the cryptp dataframes
for crypto_ob_key in crypto_ob_df_dict:
    crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key].reindex(columns=new_coloumns_lst)

    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after rearranging columns:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                


Debug BTC_5min dataframe after rearranging columns:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,0.0,1309.910034,18997.929688,8.944544e-08,2208291.0,1986.449951,1107645.375,1098659.25,-0.000129,746.049988,0.0,0.0,746.049988,0.000033,345.220001,0.0,0.0,345.220001,-0.00013,1117.859985,0.0,0.0,1117.859985,0.000278,107.75,0.0,0.0,107.75,-0.000143,2794.600098,0.0,0.0,2794.600098,0.000345,27026.910156,0.0,26445.349609,581.559998,-0.000201,14100.320312,0.0,0.0,14100.320312,0.000364,283.829987,0.0,0.0,283.829987,-0.000201,5588.879883,0.0,0.0,5588.879883,0.000421,3511.439941,312.040009,715.26001,2484.139893,-0.00024,187.220001,0.0,0.0,187.220001,0.0005,2164.22998,0.0,0.0,2164.22998,-0.000271,27942.429688,0.0,0.0,27942.429688,0.000573,19576.220703,0.0,


Debug ETH_5min dataframe after rearranging columns:


(3365, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1032.089966,0.0,0.0,1032.089966,0.000124,1163.01001,0.0,0.0,1163.01001,-0.000129,4995.910156,0.0,0.0,4995.910156,0.000129,1163.02002,0.0,0.0,1163.02002,-0.0002,29558.550781,0.0,0.0,29558.550781,0.000155,3769.629883,277.600006,3250.340088,241.690002,-0.000246,1870.839966,0.0,1673.790039,197.050003,0.000332,21908.570312,0.0,0.0,21908.570312,-0.000261,31230.509766,0.0,1673.76001,29556.75,0.000337,4873.0,0.0,0.0,4873.0,-0.000297,3241.280029,0.0,0.0,3241.280029,0.000342,77911.609375,0.0,77887.140625,24.469999,-0.000317,18661.369141,0.0,12750.349609,5911.02002,0.000348,79081.539062,0.0,79063.171875,18.370001,-0.000454,29298.880859,0.0,0.0,29298.880859,0.000353,50146.8085


Debug ADA_5min dataframe after rearranging columns:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,2461.409912,0.0,0.0,2461.409912,0.000384,9392.610352,9.62,7389.899902,1993.089966,-0.000469,2996.790039,0.0,0.0,2996.790039,0.000725,23057.310547,0.0,0.0,23057.310547,-0.00081,4068.399902,0.0,2996.97998,1071.430054,0.00081,2582.870117,0.0,808.539978,1774.329956,-0.000981,24032.550781,0.0,3474.47998,20558.070312,0.000895,34555.359375,0.0,27550.689453,7004.669922,-0.001066,1000.0,0.0,0.0,1000.0,0.000981,20860.320312,0.0,808.679993,20051.640625,-0.001663,1308.819946,0.0,0.0,1308.819946,0.001237,11549.179688,0.0,808.890015,10740.290039,-0.001748,35905.421875,0.0,25248.980469,10656.429688,0.001322,21617.269531,0.0,21134.720703,482.559998,-0.002089,3063.939941,0.0,0.0

In [9]:
# Optionally filter out "cancel" features/columns from dataframe

if (REMOVE_CANCEL_FEATURES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*cancel*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="cancel")))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering out cancels:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))

In [10]:
# Optionally filter out "market" features/columns from dataframe

if (REMOVE_MARKET_FEATURES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*market*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="market")))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering out market:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))            

In [11]:
# Filter # levels from dataframe 
# Select only the levels interested: From Level_0 --> NUM_ORDER_BOOK_LEVELS-1, drop all other levels

if (NUM_ORDER_BOOK_LEVELS < MAX_ORDER_BOOK_LEVELS):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with strings (e.g. _4, _5, _6, ...)
        for remove_level_num in range(NUM_ORDER_BOOK_LEVELS, MAX_ORDER_BOOK_LEVELS):   # Note, 1st level = level_0
            remove_level_str = "_" + str(remove_level_num)
            crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex=remove_level_str)))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering levels:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                


Debug BTC_5min dataframe after filtering levels:


(3367, 34)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,0.0,1309.910034,18997.929688,8.944544e-08,2208291.0,1986.449951,1107645.375,1098659.25,-0.000129,746.049988,0.0,0.0,746.049988,0.000033,345.220001,0.0,0.0,345.220001,-0.00013,1117.859985,0.0,0.0,1117.859985,0.000278,107.75,0.0,0.0,107.75



Debug ETH_5min dataframe after filtering levels:


(3365, 34)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1032.089966,0.0,0.0,1032.089966,0.000124,1163.01001,0.0,0.0,1163.01001,-0.000129,4995.910156,0.0,0.0,4995.910156,0.000129,1163.02002,0.0,0.0,1163.02002,-0.0002,29558.550781,0.0,0.0,29558.550781,0.000155,3769.629883,277.600006,3250.340088,241.690002



Debug ADA_5min dataframe after filtering levels:


(3367, 34)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,2461.409912,0.0,0.0,2461.409912,0.000384,9392.610352,9.62,7389.899902,1993.089966,-0.000469,2996.790039,0.0,0.0,2996.790039,0.000725,23057.310547,0.0,0.0,23057.310547,-0.00081,4068.399902,0.0,2996.97998,1071.430054,0.00081,2582.870117,0.0,808.539978,1774.329956


In [12]:
# Optionally, add actual prices at order book levels
# The provided order book data does not contain the actual price of the asset at each level, but instead provides the "delta/difference" (or distance) between the actual price and the mid price
# This will derive the actual price (mid_price + distance) and create a new column/feature in the dataframe

if (ADD_ABSOLUTE_PRICE_LEVELS):
    for crypto_ob_key in crypto_ob_df_dict:
        for level_num in range(NUM_ORDER_BOOK_LEVELS):
            for bid_ask_str in bid_ask_str_lst:
                distance_level_str = bid_ask_str + "_distance_" + str(level_num)   # Existing delta/distance price column name (e.g. bids_distance_0, bids_distance_1, ..., asks_distance_0,...)
                price_level_str = bid_ask_str + "_price_" + str(level_num)     # New column will be added to dataframe with actual prices (e.g. bids_price_0, bids_price_1, ..., asks_price_0, ...)
                # Want to insert the price column at a specific level right next to the price distance column for easy viewing (e.g. "bids_distance_0", "bids_price_0", ...)
                distance_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(distance_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(distance_column_index+1, price_level_str, (crypto_ob_df_dict[crypto_ob_key]["midpoint"] + crypto_ob_df_dict[crypto_ob_key][distance_level_str]))        

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding price level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                


Debug BTC_5min dataframe after adding price level columns:


(3367, 40)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.995,20307.839844,0.0,1309.910034,18997.929688,8.944544e-08,55899.995,2208291.0,1986.449951,1107645.375,1098659.25,-0.000129,55899.994871,746.049988,0.0,0.0,746.049988,0.000033,55899.995033,345.220001,0.0,0.0,345.220001,-0.00013,55899.99487,1117.859985,0.0,0.0,1117.859985,0.000278,55899.995278,107.75,0.0,0.0,107.75



Debug ETH_5min dataframe after adding price level columns:


(3365, 40)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1970.964876,1032.089966,0.0,0.0,1032.089966,0.000124,1970.965124,1163.01001,0.0,0.0,1163.01001,-0.000129,1970.964871,4995.910156,0.0,0.0,4995.910156,0.000129,1970.965129,1163.02002,0.0,0.0,1163.02002,-0.0002,1970.9648,29558.550781,0.0,0.0,29558.550781,0.000155,1970.965155,3769.629883,277.600006,3250.340088,241.690002



Debug ADA_5min dataframe after adding price level columns:


(3367, 40)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,1.172166,2461.409912,0.0,0.0,2461.409912,0.000384,1.172934,9392.610352,9.62,7389.899902,1993.089966,-0.000469,1.172081,2996.790039,0.0,0.0,2996.790039,0.000725,1.173275,23057.310547,0.0,0.0,23057.310547,-0.00081,1.17174,4068.399902,0.0,2996.97998,1071.430054,0.00081,1.17336,2582.870117,0.0,808.539978,1774.329956


In [13]:
# Optionally, filter out price distance features (e.g. bids_distance_0, bids_distance_1, ..., asks_distance_0,...)

if (REMOVE_PRICE_DISTANCES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*distance*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="_distance_")))]
        
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after dropping distance column:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))        

In [14]:
# Add the quantity at each level
# The downloaded crypo order book provides the "notional" value which equals the price (at that level) * the quantity (at the level)
# Derive the quantiy by computing: notional / price

# There are 4 notional fields provided at each level: notional, limit_notional, market_notional, and cancel_notional
# where "notional = limit_notional - market_notional - cancel_notional"
notional_str_lst = ["_limit_"]
if (not REMOVE_MARKET_FEATURES): 
    notional_str_lst.append("_market_")
if (not REMOVE_CANCEL_FEATURES): 
    notional_str_lst.append("_cancel_")
notional_str_lst.append("_")    # For the "notiona" column by itself

for crypto_ob_key in crypto_ob_df_dict:
    for level_num in range(NUM_ORDER_BOOK_LEVELS):
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                notional_level_str = bid_ask_str + notional_str + "notional" + "_" + str(level_num)
                price_level_str = bid_ask_str + "_price" + "_" + str(level_num)
                quantity_level_str = bid_ask_str + notional_str + "quantity" + "_" + str(level_num)
                # Want to insert the quantity column at a specific level right before the notional column for easy viewing (e.g. "bids_quantity_0", "bids_notional_0", ...)
                notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(notional_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index, quantity_level_str, (crypto_ob_df_dict[crypto_ob_key][notional_level_str] / crypto_ob_df_dict[crypto_ob_key][price_level_str]))   
                #print(f"{notional_level_str}, {price_level_str}, {quantity_level_str}")
                
    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after adding the quantity feature column:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))            


Debug BTC_5min dataframe after adding the quantity feature column:


(3367, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.995,0.363289,20307.839844,0.0,0.0,0.023433,1309.910034,0.339856,18997.929688,8.944544e-08,55899.995,39.504315,2208291.0,0.035536,1986.449951,19.814767,1107645.375,19.654013,1098659.25,-0.000129,55899.994871,0.013346,746.049988,0.0,0.0,0.0,0.0,0.013346,746.049988,0.000033,55899.995033,0.006176,345.220001,0.0,0.0,0.0,0.0,0.006176,345.220001,-0.00013,55899.99487,0.019997,1117.859985,0.0,0.0,0.0,0.0,0.019997,1117.859985,0.000278,55899.995278,0.001928,107.75,0.0,0.0,0.0,0.0,0.001928,107.75



Debug ETH_5min dataframe after adding the quantity feature column:


(3365, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1970.964876,0.523647,1032.089966,0.0,0.0,0.0,0.0,0.523647,1032.089966,0.000124,1970.965124,0.590071,1163.01001,0.0,0.0,0.0,0.0,0.590071,1163.01001,-0.000129,1970.964871,2.534754,4995.910156,0.0,0.0,0.0,0.0,2.534754,4995.910156,0.000129,1970.965129,0.590076,1163.02002,0.0,0.0,0.0,0.0,0.590076,1163.02002,-0.0002,1970.9648,14.996996,29558.550781,0.0,0.0,0.0,0.0,14.996996,29558.550781,0.000155,1970.965155,1.912581,3769.629883,0.140845,277.600006,1.649111,3250.340088,0.122625,241.690002



Debug ADA_5min dataframe after adding the quantity feature column:


(3367, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,1.172166,2099.881286,2461.409912,0.0,0.0,0.0,0.0,2099.881286,2461.409912,0.000384,1.172934,8007.792529,9392.610352,8.201656,9.62,6300.3556,7389.899902,1699.23486,1993.089966,-0.000469,1.172081,2556.811518,2996.790039,0.0,0.0,0.0,0.0,2556.811518,2996.790039,0.000725,1.173275,19652.095376,23057.310547,0.0,0.0,0.0,0.0,19652.095376,23057.310547,-0.00081,1.17174,3472.101829,4068.399902,0.0,0.0,2557.718002,2996.97998,914.392473,1071.430054,0.00081,1.17336,2201.259355,2582.870117,0.0,0.0,689.080794,808.539978,1512.178405,1774.329956


In [15]:
# Add the cumulative quantity at each level
# Example. cum_quantity_0 = quantity_0
#          cum_quantity_1 = quantity_0 + quantity_1
#          cum_quantity_2 = quantity_0 + quantity_1 + quantity_2
#          ....

if (ADD_CUM_QUANTITY_FEATURE):
    for crypto_ob_key in crypto_ob_df_dict:
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                quantity_level_df = pd.DataFrame()    # empty dataframe to hold all the values of a specific type of quantity (e.g. bids_limit_quantity)
                                                      # at each level (e.g. bids_limit_quantity_0, bids_limit_quantity_1, bids_limit_quantity_2, ....)                
                for level_num in range(NUM_ORDER_BOOK_LEVELS):
                    quantity_level_str = bid_ask_str + notional_str + "quantity" + "_" + str(level_num)
                    quantity_level_df[quantity_level_str] = crypto_ob_df_dict[crypto_ob_key][quantity_level_str]
                    # Compute the cumulative quantity across columns (axis=1) of the same row 
                    cum_quantity_level_df = quantity_level_df.cumsum(axis = 1)
                    
                    # Copy the the cum quantity data frame to the main crypto data frame
                    # Want to insert the cum quantity column at a specific level right next to the quantity column for easy viewing (e.g. "bids_quantity_0", "bids_cum_quantity_0", ...)
                    quantity_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(quantity_level_str)
                    cum_quantity_level_str = bid_ask_str + notional_str + "cum_quantity" + "_" + str(level_num)
                    crypto_ob_df_dict[crypto_ob_key].insert(quantity_column_index+1, cum_quantity_level_str, cum_quantity_level_df[quantity_level_str])
                    
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding cum quantity level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                                    


Debug BTC_5min dataframe after adding cum quantity level columns:


(3367, 88)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.995,0.363289,0.363289,20307.839844,0.0,0.0,0.0,0.023433,0.023433,1309.910034,0.339856,0.339856,18997.929688,8.944544e-08,55899.995,39.504315,39.504315,2208291.0,0.035536,0.035536,1986.449951,19.814767,19.814767,1107645.375,19.654013,19.654013,1098659.25,-0.000129,55899.994871,0.013346,0.376635,746.049988,0.0,0.0,0.0,0.0,0.023433,0.0,0.013346,0.353202,746.049988,0.000033,55899.995033,0.006176,39.51049,345.220001,0.0,0.035536,0.0,0.0,19.814767,0.0,0.006176,19.660189,345.220001,-0.00013,55899.99487,0.019997,0.396632,1117.859985,0.0,0.0,0.0,0.0,0.023433,0.0,0.019997,0.373199,1117.859985,0.000278,55899.995278,0.001928,39.512418,107.75,0.0,0.035536,0.0,0.0,19.814767,0.0,0.001928,19.662117,107.75



Debug ETH_5min dataframe after adding cum quantity level columns:


(3365, 88)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1970.964876,0.523647,0.523647,1032.089966,0.0,0.0,0.0,0.0,0.0,0.0,0.523647,0.523647,1032.089966,0.000124,1970.965124,0.590071,0.590071,1163.01001,0.0,0.0,0.0,0.0,0.0,0.0,0.590071,0.590071,1163.01001,-0.000129,1970.964871,2.534754,3.058401,4995.910156,0.0,0.0,0.0,0.0,0.0,0.0,2.534754,3.058401,4995.910156,0.000129,1970.965129,0.590076,1.180148,1163.02002,0.0,0.0,0.0,0.0,0.0,0.0,0.590076,1.180148,1163.02002,-0.0002,1970.9648,14.996996,18.055396,29558.550781,0.0,0.0,0.0,0.0,0.0,0.0,14.996996,18.055396,29558.550781,0.000155,1970.965155,1.912581,3.092728,3769.629883,0.140845,0.140845,277.600006,1.649111,1.649111,3250.340088,0.122625,1.302773,241.690002



Debug ADA_5min dataframe after adding cum quantity level columns:


(3367, 88)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,1.172166,2099.881286,2099.881286,2461.409912,0.0,0.0,0.0,0.0,0.0,0.0,2099.881286,2099.881286,2461.409912,0.000384,1.172934,8007.792529,8007.792529,9392.610352,8.201656,8.201656,9.62,6300.3556,6300.3556,7389.899902,1699.23486,1699.23486,1993.089966,-0.000469,1.172081,2556.811518,4656.692804,2996.790039,0.0,0.0,0.0,0.0,0.0,0.0,2556.811518,4656.692804,2996.790039,0.000725,1.173275,19652.095376,27659.887905,23057.310547,0.0,8.201656,0.0,0.0,6300.3556,0.0,19652.095376,21351.330236,23057.310547,-0.00081,1.17174,3472.101829,8128.794633,4068.399902,0.0,0.0,0.0,2557.718002,2557.718002,2996.97998,914.392473,5571.085277,1071.430054,0.00081,1.17336,2201.259355,29861.14726,2582.870117,0.0,8.201656,0.0,689.080794,6989.436394,808.539978,1512.178405,22863.508641,1774.329956


In [16]:
# Add the cumulative notional at each level
# Example. cum_notional_0 = notional_0
#          cum_notional_1 = notional_0 + notional_1
#          cum_notional_2 = notional_0 + notional_1 + notional_2
#          ....

if (ADD_CUM_NOTIONAL_FEATURE):
    for crypto_ob_key in crypto_ob_df_dict:
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                notional_level_df = pd.DataFrame()    # empty dataframe to hold all the values of a specific type of notional (e.g. bids_limit_notional)
                                                      # at each level (e.g. bids_limit_notional_0, bids_limit_notional_1, bids_limit_notional_2, ....)                
                for level_num in range(NUM_ORDER_BOOK_LEVELS):
                    notional_level_str = bid_ask_str + notional_str + "notional" + "_" + str(level_num)
                    notional_level_df[notional_level_str] = crypto_ob_df_dict[crypto_ob_key][notional_level_str]
                    # Compute the cumulative notional across columns (axis=1) of the same row 
                    cum_notional_level_df = notional_level_df.cumsum(axis = 1)
                    
                    # Copy the the cum notional data frame to the main crypto data frame
                    # Want to insert the cum notional column at a specific level right next to the notional column for easy viewing (e.g. "bids_notional_0", "bids_cum_notional_0", ...)
                    notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(notional_level_str)
                    cum_notional_level_str = bid_ask_str + notional_str + "cum_notional" + "_" + str(level_num)
                    crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index+1, cum_notional_level_str, cum_notional_level_df[notional_level_str])
                    
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding cum notional level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1)) 


Debug BTC_5min dataframe after adding cum notional level columns:


(3367, 112)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.995,0.363289,0.363289,20307.839844,20307.839844,0.0,0.0,0.0,0.0,0.023433,0.023433,1309.910034,1309.910034,0.339856,0.339856,18997.929688,18997.929688,8.944544e-08,55899.995,39.504315,39.504315,2208291.0,2208291.0,0.035536,0.035536,1986.449951,1986.449951,19.814767,19.814767,1107645.375,1107645.375,19.654013,19.654013,1098659.25,1098659.25,-0.000129,55899.994871,0.013346,0.376635,746.049988,21053.889832,0.0,0.0,0.0,0.0,0.0,0.023433,0.0,1309.910034,0.013346,0.353202,746.049988,19743.979675,0.000033,55899.995033,0.006176,39.51049,345.220001,2.208636e+06,0.0,0.035536,0.0,1986.449951,0.0,19.814767,0.0,1107645.375,0.006176,19.660189,345.220001,1.099004e+06,-0.00013,55899.99487,0.019997,0.396632,1117.859985,22171.749817,0.0,0.0,0.0,0.0,0.0,0.023433,0.0,1309.910034,0.019997,0.373199,1117.859985,20861.839661,0.000278,55899.995278,0.001928,39.512418,107.75,2.208744e+06,0.0,0.035536,0.0,1986.449951,0.0,19.814767,0.0,1107645.375,0.001928,19.662117,107.75,1.099112e+06



Debug ETH_5min dataframe after adding cum notional level columns:


(3365, 112)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1970.964876,0.523647,0.523647,1032.089966,1032.089966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.523647,0.523647,1032.089966,1032.089966,0.000124,1970.965124,0.590071,0.590071,1163.01001,1163.01001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.590071,0.590071,1163.01001,1163.01001,-0.000129,1970.964871,2.534754,3.058401,4995.910156,6028.000122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.534754,3.058401,4995.910156,6028.000122,0.000129,1970.965129,0.590076,1.180148,1163.02002,2326.030029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.590076,1.180148,1163.02002,2326.030029,-0.0002,1970.9648,14.996996,18.055396,29558.550781,35586.550903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.996996,18.055396,29558.550781,35586.550903,0.000155,1970.965155,1.912581,3.092728,3769.629883,6095.659912,0.140845,0.140845,277.600006,277.600006,1.649111,1.649111,3250.340088,3250.340088,0.122625,1.302773,241.690002,2567.720032



Debug ADA_5min dataframe after adding cum notional level columns:


(3367, 112)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,1.172166,2099.881286,2099.881286,2461.409912,2461.409912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2099.881286,2099.881286,2461.409912,2461.409912,0.000384,1.172934,8007.792529,8007.792529,9392.610352,9392.610352,8.201656,8.201656,9.62,9.62,6300.3556,6300.3556,7389.899902,7389.899902,1699.23486,1699.23486,1993.089966,1993.089966,-0.000469,1.172081,2556.811518,4656.692804,2996.790039,5458.199951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2556.811518,4656.692804,2996.790039,5458.199951,0.000725,1.173275,19652.095376,27659.887905,23057.310547,32449.920898,0.0,8.201656,0.0,9.62,0.0,6300.3556,0.0,7389.899902,19652.095376,21351.330236,23057.310547,25050.400513,-0.00081,1.17174,3472.101829,8128.794633,4068.399902,9526.599854,0.0,0.0,0.0,0.0,2557.718002,2557.718002,2996.97998,2996.97998,914.392473,5571.085277,1071.430054,6529.630005,0.00081,1.17336,2201.259355,29861.14726,2582.870117,35032.791016,0.0,8.201656,0.0,9.62,689.080794,6989.436394,808.539978,8198.43988,1512.178405,22863.508641,1774.329956,26824.730469


In [17]:
# Add the bid-ask imbalance at each level
# Imbalance = bid_notional / (bid_notional + ask_notional)
#    < 0.5, more buyers than sellers
#    > 0.5, more sellers than buyers
#    How may this move the prices?

if (ADD_BID_ASK_IMBALANCE):
    for crypto_ob_key in crypto_ob_df_dict:
        for level_num in range(NUM_ORDER_BOOK_LEVELS):
            for notional_str in notional_str_lst:
                bid_notional_level_str = "bids" + notional_str + "notional" + "_" + str(level_num)
                ask_notional_level_str = "asks" + notional_str + "notional" + "_" + str(level_num)            
                imbalance_level_str = "imbalance" + notional_str + "notional" + "_" + str(level_num)            
            
                # Want to insert the quantity column at a specific level right after the ask notional column for easy viewing
                notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(ask_notional_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index+1, imbalance_level_str, (crypto_ob_df_dict[crypto_ob_key][bid_notional_level_str] / (crypto_ob_df_dict[crypto_ob_key][bid_notional_level_str] + crypto_ob_df_dict[crypto_ob_key][ask_notional_level_str])))   
                
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding the quantity feature column:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))           


Debug BTC_5min dataframe after adding the quantity feature column:


(3367, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,imbalance_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,imbalance_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,imbalance_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,imbalance_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,imbalance_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,imbalance_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,imbalance_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,imbalance_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,imbalance_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,imbalance_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,imbalance_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,imbalance_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.995,0.363289,0.363289,20307.839844,20307.839844,0.0,0.0,0.0,0.0,0.023433,0.023433,1309.910034,1309.910034,0.339856,0.339856,18997.929688,18997.929688,8.944544e-08,55899.995,39.504315,39.504315,2208291.0,0.009112,2208291.0,0.035536,0.035536,1986.449951,0.0,1986.449951,19.814767,19.814767,1107645.375,0.001181,1107645.375,19.654013,19.654013,1098659.25,0.016998,1098659.25,-0.000129,55899.994871,0.013346,0.376635,746.049988,21053.889832,0.0,0.0,0.0,0.0,0.0,0.023433,0.0,1309.910034,0.013346,0.353202,746.049988,19743.979675,0.000033,55899.995033,0.006176,39.51049,345.220001,0.683653,2.208636e+06,0.0,0.035536,0.0,NaN,1986.449951,0.0,19.814767,0.0,NaN,1107645.375,0.006176,19.660189,345.220001,0.683653,1.099004e+06,-0.00013,55899.99487,0.019997,0.396632,1117.859985,22171.749817,0.0,0.0,0.0,0.0,0.0,0.023433,0.0,1309.910034,0.019997,0.373199,1117.859985,20861.839661,0.000278,55899.995278,0.001928,39.512418,107.75,0.912085,2.208744e+06,0.0,0.035536,0.0,NaN,1986.449951,0.0,19.814767,0.0,NaN,1107645.375,0.001928,19.662117,107.75,0.912085,1.099112


Debug ETH_5min dataframe after adding the quantity feature column:


(3365, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,imbalance_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,imbalance_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,imbalance_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,imbalance_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,imbalance_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,imbalance_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,imbalance_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,imbalance_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,imbalance_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,imbalance_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,imbalance_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,imbalance_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:49.861733+00:00,1970.965,0.49,3.160851e+06,5.002265e+06,-0.000124,1970.964876,0.523647,0.523647,1032.089966,1032.089966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.523647,0.523647,1032.089966,1032.089966,0.000124,1970.965124,0.590071,0.590071,1163.01001,0.470179,1163.01001,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.590071,0.590071,1163.01001,0.470179,1163.01001,-0.000129,1970.964871,2.534754,3.058401,4995.910156,6028.000122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.534754,3.058401,4995.910156,6028.000122,0.000129,1970.965129,0.590076,1.180148,1163.02002,0.811165,2326.030029,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.590076,1.180148,1163.02002,0.811165,2326.030029,-0.0002,1970.9648,14.996996,18.055396,29558.550781,35586.550903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.996996,18.055396,29558.550781,35586.550903,0.000155,1970.965155,1.912581,3.092728,3769.629883,0.886894,6095.659912,0.140845,0.140845,277.600006,0.0,277.600006,1.649111,1.649111,3250.340088,0.0,3250.340088,0.122625,1.302773,241.690002,0.99189,2567.720032



Debug ADA_5min dataframe after adding the quantity feature column:


(3367, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,bids_market_quantity_0,bids_market_cum_quantity_0,bids_market_notional_0,bids_market_cum_notional_0,bids_cancel_quantity_0,bids_cancel_cum_quantity_0,bids_cancel_notional_0,bids_cancel_cum_notional_0,bids_quantity_0,bids_cum_quantity_0,bids_notional_0,bids_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,imbalance_limit_notional_0,asks_limit_cum_notional_0,asks_market_quantity_0,asks_market_cum_quantity_0,asks_market_notional_0,imbalance_market_notional_0,asks_market_cum_notional_0,asks_cancel_quantity_0,asks_cancel_cum_quantity_0,asks_cancel_notional_0,imbalance_cancel_notional_0,asks_cancel_cum_notional_0,asks_quantity_0,asks_cum_quantity_0,asks_notional_0,imbalance_notional_0,asks_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,bids_market_quantity_1,bids_market_cum_quantity_1,bids_market_notional_1,bids_market_cum_notional_1,bids_cancel_quantity_1,bids_cancel_cum_quantity_1,bids_cancel_notional_1,bids_cancel_cum_notional_1,bids_quantity_1,bids_cum_quantity_1,bids_notional_1,bids_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,imbalance_limit_notional_1,asks_limit_cum_notional_1,asks_market_quantity_1,asks_market_cum_quantity_1,asks_market_notional_1,imbalance_market_notional_1,asks_market_cum_notional_1,asks_cancel_quantity_1,asks_cancel_cum_quantity_1,asks_cancel_notional_1,imbalance_cancel_notional_1,asks_cancel_cum_notional_1,asks_quantity_1,asks_cum_quantity_1,asks_notional_1,imbalance_notional_1,asks_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,bids_market_quantity_2,bids_market_cum_quantity_2,bids_market_notional_2,bids_market_cum_notional_2,bids_cancel_quantity_2,bids_cancel_cum_quantity_2,bids_cancel_notional_2,bids_cancel_cum_notional_2,bids_quantity_2,bids_cum_quantity_2,bids_notional_2,bids_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,imbalance_limit_notional_2,asks_limit_cum_notional_2,asks_market_quantity_2,asks_market_cum_quantity_2,asks_market_notional_2,imbalance_market_notional_2,asks_market_cum_notional_2,asks_cancel_quantity_2,asks_cancel_cum_quantity_2,asks_cancel_notional_2,imbalance_cancel_notional_2,asks_cancel_cum_notional_2,asks_quantity_2,asks_cum_quantity_2,asks_notional_2,imbalance_notional_2,asks_cum_notional_2
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:59.055697+00:00,1.17255,0.0009,330639.048591,500585.424671,-0.000384,1.172166,2099.881286,2099.881286,2461.409912,2461.409912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2099.881286,2099.881286,2461.409912,2461.409912,0.000384,1.172934,8007.792529,8007.792529,9392.610352,0.207643,9392.610352,8.201656,8.201656,9.62,0.0,9.62,6300.3556,6300.3556,7389.899902,0.0,7389.899902,1699.23486,1699.23486,1993.089966,0.552567,1993.089966,-0.000469,1.172081,2556.811518,4656.692804,2996.790039,5458.199951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2556.811518,4656.692804,2996.790039,5458.199951,0.000725,1.173275,19652.095376,27659.887905,23057.310547,0.115022,32449.920898,0.0,8.201656,0.0,NaN,9.62,0.0,6300.3556,0.0,NaN,7389.899902,19652.095376,21351.330236,23057.310547,0.115022,25050.400513,-0.00081,1.17174,3472.101829,8128.794633,4068.399902,9526.599854,0.0,0.0,0.0,0.0,2557.718002,2557.718002,2996.97998,2996.97998,914.392473,5571.085277,1071.430054,6529.630005,0.00081,1.17336,2201.259355,29861.14726,2582.870117,0.611673,35032.791016,0.0,8.201656,0.0,NaN,9.62,689.080794,6989.436394,808.539978,0.787535,8198.43988,1512.178405,22863.508641,1774.3299

In [18]:
# Save resultant crypto dataframe to be used in machine learning models to csv
for crypto_ob_key in crypto_ob_df_dict:
    csv_file = "./derived_data/crypto_" + crypto_ob_key + ".csv"
    crypto_ob_df_dict[crypto_ob_key].to_csv(csv_file, sep=',', index=True,header=True)